In [1]:
'''

from google.colab import drive
drive.mount('/content/drive')

# Crée un dossier dans ton Drive si tu veux
%cd /content/drive/MyDrive/training_dataset/src_2/

'''



"\n\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\n# Crée un dossier dans ton Drive si tu veux\n%cd /content/drive/MyDrive/training_dataset/src_2/\n\n"

Importations

In [ ]:
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install Roboflow

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Download dataset from roboflow api key

In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="7cxew9iyBbltgkRKqUJT")
project = rf.workspace("myworkspace-8fngg").project("dataset_1-ze0og")
version = project.version(9)
dataset = version.download("yolov8")

/Users/valeriecowan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


loading Roboflow workspace...
loading Roboflow project...


In [5]:
'''
from roboflow import Roboflow
rf = Roboflow(api_key="7cxew9iyBbltgkRKqUJT")
project = rf.workspace("myworkspace-8fngg").project("dataset_1-ze0og")
version = project.version(8)
dataset = version.download("yolov8")
'''

'\nfrom roboflow import Roboflow\nrf = Roboflow(api_key="7cxew9iyBbltgkRKqUJT")\nproject = rf.workspace("myworkspace-8fngg").project("dataset_1-ze0og")\nversion = project.version(8)\ndataset = version.download("yolov8")\n'

In [6]:
ls

Dataset_1-8/               run_yolo_training-2.ipynb
Dataset_1-9/               run_yolo_training.ipynb
file.ipynb                 runs/
modified_yaml.yaml         yolov8n.pt


In [7]:
%cd Dataset_1-9/

/Users/valeriecowan/Desktop/Mathieu/Dataset_1-9


using dhist requires you to install the `pickleshare` library.


In [8]:
ls

README.dataset.txt          train/
README.roboflow.txt         valid/
data.yaml                   yaml_initial_roboflow.yaml
test/


In [ ]:
import yaml
import os

# 1. afficher le contenu de data.yaml
print("Contenu de data.yaml :\n")
with open("data.yaml", "r") as f:
    data_yaml = yaml.safe_load(f)
    print(yaml.dump(data_yaml, sort_keys=False))




Contenu de data.yaml :

nc: 80
names:
  0: person
  1: bicycle
  2: car
  3: motorcycle
  4: airplane
  5: bus
  6: train
  7: truck
  8: boat
  9: traffic light
  10: fire hydrant
  11: stop sign
  12: parking meter
  13: bench
  14: bird
  15: cat
  16: dog
  17: horse
  18: sheep
  19: cow
  20: elephant
  21: bear
  22: zebra
  23: giraffe
  24: backpack
  25: umbrella
  26: handbag
  27: tie
  28: suitcase
  29: frisbee
  30: skis
  31: snowboard
  32: sports ball
  33: kite
  34: baseball bat
  35: baseball glove
  36: skateboard
  37: surfboard
  38: tennis racket
  39: bottle
  40: wine glass
  41: cup
  42: fork
  43: knife
  44: spoon
  45: bowl
  46: banana
  47: apple
  48: sandwich
  49: orange
  50: barrier
  51: carrot
  52: post
  53: cake
  54: chair
  55: obstacle
  56: couch
  57: potted plant
  58: motorcycle
  59: bed
  60: dining table
  61: toilet
  62: tv
  63: tree
  64: laptop
  65: mouse
  66: remote
  67: keyboard
  68: cell phone
  69: microwave
  70: oven


update the indexes to replace unwanted classes in the YOLO model (see the custom_map to see the exact changes)

In [ ]:
# 2. afficher le contenu du premier .txt dans test/labels/
print("\nContenu du premier .txt dans test/labels/ :\n")
label_dir = "test/labels"
txt_files = [f for f in os.listdir(label_dir) if f.endswith(".txt")]

if txt_files:
    first_file = txt_files[0]
    print(f"Fichier : {first_file}\n")
    with open(os.path.join(label_dir, first_file), "r") as f:
        print(f.read())
else:
    print("Aucun fichier .txt trouvé dans test/labels/")

Index mapping

In [10]:
import os

# Mapping Roboflow index → COCO index
custom_map = {
    0: 50,  # barrier
    1: 58,  # motorcycle
    2: 55,  # obstacle
    3: 77,  # person (custom)
    4: 52,  # post
    5: 63,  # tree
    6: 73,  # vehicule
    7: 76   # wall
}

def remap_labels_in_folder(label_dir):
    for fname in os.listdir(label_dir):
        if fname.endswith(".txt"):
            fpath = os.path.join(label_dir, fname)
            with open(fpath, 'r') as f:
                lines = f.readlines()

            new_lines = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                old_idx = int(parts[0])
                if old_idx in custom_map:
                    parts[0] = str(custom_map[old_idx])
                    new_lines.append(' '.join(parts))

            with open(fpath, 'w') as f:
                f.write('\n'.join(new_lines))

# Appliquer sur train/labels, valid/labels, test/labels
for split in ['train', 'valid', 'test']:
    label_path = f"{split}/labels"
    if os.path.exists(label_path):
        remap_labels_in_folder(label_path)


In [11]:
print("\nContenu du premier .txt dans test/labels/ :\n")
label_dir = "test/labels"
txt_files = [f for f in os.listdir(label_dir) if f.endswith(".txt")]

if txt_files:
    first_file = txt_files[0]
    print(f"Fichier : {first_file}\n")
    with open(os.path.join(label_dir, first_file), "r") as f:
        print(f.read())
else:
    print("Aucun fichier .txt trouvé dans test/labels/")


Contenu du premier .txt dans test/labels/ :

Fichier : video_1_frame_130_jpg.rf.c48ae9e3aac46193bc26691efaf31c93.txt




In [12]:
#check corrections of indexes
import os

label_folders = ["train/labels", "valid/labels", "test/labels"]
expected_indices = {50, 52, 55, 58, 63, 73, 76, 77}

for folder in label_folders:
    print(f"\n📂 Vérification dans {folder}:")
    for fname in os.listdir(folder):
        if fname.endswith(".txt"):
            with open(os.path.join(folder, fname)) as f:
                lines = f.readlines()
                for i, line in enumerate(lines):
                    label_idx = int(line.split()[0])
                    if label_idx not in expected_indices:
                        print(f"⚠️ {fname} ligne {i+1} → index {label_idx} inattendu")



📂 Vérification dans train/labels:

📂 Vérification dans valid/labels:

📂 Vérification dans test/labels:


if no logs, there are no anomalies

In [13]:
ls

README.dataset.txt          train/
README.roboflow.txt         valid/
data.yaml                   yaml_initial_roboflow.yaml
test/


In [14]:
%cd ../

/Users/valeriecowan/Desktop/Mathieu


In [15]:
ls

Dataset_1-8/               run_yolo_training-2.ipynb
Dataset_1-9/               run_yolo_training.ipynb
file.ipynb                 runs/
modified_yaml.yaml         yolov8n.pt


In [16]:
!ls -lh yolov8n.pt



-rw-r--r--@ 1 valeriecowan  staff   6.2M May 18 12:13 yolov8n.pt


There was a pb with yolo we redownload it

In [17]:
import requests

url = "https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt"
output_path = "yolov8n.pt"

response = requests.get(url)
with open(output_path, "wb") as f:
    f.write(response.content)

print("Téléchargement terminé.")



Téléchargement terminé.


In [18]:
!ls -lh yolov8n.pt

-rw-r--r--@ 1 valeriecowan  staff   6.2M May 18 15:55 yolov8n.pt


Vérification de la cohérence des labels/classes avant de faire l'entrainement

In [19]:
ls

Dataset_1-8/               run_yolo_training-2.ipynb
Dataset_1-9/               run_yolo_training.ipynb
file.ipynb                 runs/
modified_yaml.yaml         yolov8n.pt


In [20]:
# Lecture et affichage du contenu du YAML
yaml_file = "Dataset_1-9/data.yaml"

with open(yaml_file, "r", encoding="utf-8") as f:
    content = f.read()

print(content)


nc: 80

names:
  0: person
  1: bicycle
  2: car
  3: motorcycle
  4: airplane
  5: bus
  6: train
  7: truck
  8: boat
  9: traffic light
  10: fire hydrant
  11: stop sign
  12: parking meter
  13: bench
  14: bird
  15: cat
  16: dog
  17: horse
  18: sheep
  19: cow
  20: elephant
  21: bear
  22: zebra
  23: giraffe
  24: backpack
  25: umbrella
  26: handbag
  27: tie
  28: suitcase
  29: frisbee
  30: skis
  31: snowboard
  32: sports ball
  33: kite
  34: baseball bat
  35: baseball glove
  36: skateboard
  37: surfboard
  38: tennis racket
  39: bottle
  40: wine glass
  41: cup
  42: fork
  43: knife
  44: spoon
  45: bowl
  46: banana
  47: apple
  48: sandwich
  49: orange
  50: barrier
  51: carrot
  52: post
  53: cake
  54: chair
  55: obstacle
  56: couch
  57: potted plant
  58: motorcycle
  59: bed
  60: dining table
  61: toilet
  62: tv
  63: tree
  64: laptop
  65: mouse
  66: remote
  67: keyboard
  68: cell phone
  69: microwave
  70: oven
  71: toaster
  72: sin

In [21]:
import os

yaml_file = "Dataset_1-9/data.yaml"
backup_file = "Dataset_1-9/yaml_initial_roboflow.yaml"

replacement_text = """\
nc: 80

names:
  0: person
  1: bicycle
  2: car
  3: motorcycle
  4: airplane
  5: bus
  6: train
  7: truck
  8: boat
  9: traffic light
  10: fire hydrant
  11: stop sign
  12: parking meter
  13: bench
  14: bird
  15: cat
  16: dog
  17: horse
  18: sheep
  19: cow
  20: elephant
  21: bear
  22: zebra
  23: giraffe
  24: backpack
  25: umbrella
  26: handbag
  27: tie
  28: suitcase
  29: frisbee
  30: skis
  31: snowboard
  32: sports ball
  33: kite
  34: baseball bat
  35: baseball glove
  36: skateboard
  37: surfboard
  38: tennis racket
  39: bottle
  40: wine glass
  41: cup
  42: fork
  43: knife
  44: spoon
  45: bowl
  46: banana
  47: apple
  48: sandwich
  49: orange
  50: barrier
  51: carrot
  52: post
  53: cake
  54: chair
  55: obstacle
  56: couch
  57: potted plant
  58: motorcycle
  59: bed
  60: dining table
  61: toilet
  62: tv
  63: tree
  64: laptop
  65: mouse
  66: remote
  67: keyboard
  68: cell phone
  69: microwave
  70: oven
  71: toaster
  72: sink
  73: vehicule
  74: refrigerator
  75: book
  76: wall
  77: person
  78: clock
  79: vase
"""

# Lire le YAML original
with open(yaml_file, "r", encoding="utf-8") as f:
    original_content = f.read()

# Créer le backup uniquement s’il n’existe pas déjà
if not os.path.exists(backup_file):
    with open(backup_file, "w", encoding="utf-8") as f:
        f.write(original_content)
    print(f"Backup créé : '{backup_file}'")
else:
    print(f"Backup existant trouvé : '{backup_file}' - sauvegarde non écrasée")

# Trouver l’index de la ligne 'roboflow:'
lines = original_content.splitlines(keepends=True)
start_idx = None
for i, line in enumerate(lines):
    if line.strip().startswith("roboflow:"):
        start_idx = i
        break

if start_idx is None:
    raise ValueError("La clé 'roboflow:' n'a pas été trouvée dans le fichier YAML.")

# Construire le nouveau contenu YAML
new_content = replacement_text.rstrip() + "\n\n" + "".join(lines[start_idx:])

# Écrire dans le fichier YAML original
with open(yaml_file, "w", encoding="utf-8") as f:
    f.write(new_content)

print(f"Fichier '{yaml_file}' mis à jour avec la nouvelle configuration.")


Backup existant trouvé : 'Dataset_1-9/yaml_initial_roboflow.yaml' - sauvegarde non écrasée
Fichier 'Dataset_1-9/data.yaml' mis à jour avec la nouvelle configuration.


In [22]:
ls

Dataset_1-8/               run_yolo_training-2.ipynb
Dataset_1-9/               run_yolo_training.ipynb
file.ipynb                 runs/
modified_yaml.yaml         yolov8n.pt


In [23]:
%cd Dataset_1-9/

/Users/valeriecowan/Desktop/Mathieu/Dataset_1-9


In [ ]:
import os

def check_labels(folder_paths, max_class_index=79):
    errors_found = False

    for folder in folder_paths:
        label_dir = os.path.join(folder, "labels")
        if not os.path.exists(label_dir):
            print(f"⚠️ Le dossier {label_dir} n'existe pas.")
            continue

        print(f"\n🔍 Vérification des labels dans : {label_dir}")
        for fname in os.listdir(label_dir):
            if not fname.endswith(".txt"):
                continue
            fpath = os.path.join(label_dir, fname)
            try:
                with open(fpath, "r") as f:
                    lines = f.readlines()
            except Exception as e:
                print(f"Impossible de lire {fpath}: {e}")
                errors_found = True
                continue

            for i, line in enumerate(lines):
                parts = line.strip().split()
                if len(parts) != 5:
                    print(f"Format incorrect dans {fname} ligne {i+1}: '{line.strip()}' (attendu: 5 éléments)")
                    errors_found = True
                    continue
                try:
                    cls = int(parts[0])
                    if cls < 0 or cls > max_class_index:
                        print(f"Classe invalide dans {fname} ligne {i+1}: {cls} (doit être entre 0 et {max_class_index})")
                        errors_found = True
                except ValueError:
                    print(f"Classe non entière dans {fname} ligne {i+1}: '{parts[0]}'")
                    errors_found = True

    if not errors_found:
        print("\nTous les fichiers de labels sont corrects.")
    else:
        print("\nErreurs détectées dans les fichiers de labels, merci de corriger avant l'entraînement.")

# Appel de la fonction
check_labels(["train", "valid", "test"], max_class_index=79)



🔍 Vérification des labels dans : train/labels

🔍 Vérification des labels dans : valid/labels

🔍 Vérification des labels dans : test/labels

✅ Tous les fichiers de labels sont corrects.


In [25]:
%cd ../

/Users/valeriecowan/Desktop/Mathieu


In [26]:
ls

Dataset_1-8/               run_yolo_training-2.ipynb
Dataset_1-9/               run_yolo_training.ipynb
file.ipynb                 runs/
modified_yaml.yaml         yolov8n.pt


In [27]:
#pip install --upgrade --force-reinstall numpy


In [28]:
import torch
import numpy

print("PyTorch version:", torch.__version__)
print("NumPy version:", numpy.__version__)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/valeriecowan/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/User

PyTorch version: 2.2.2
NumPy version: 2.0.2


In [29]:
#train
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # Le modèle est ici dans src/

# Lancer l'entraînement en pointant vers le data.yaml dans dataset/second_dataset_od-8
model.train(
    data="Dataset_1-9/data.yaml",  # chemin relatif depuis src/
    epochs=50,
    imgsz=640,
    batch=16
)


New https://pypi.org/project/ultralytics/8.3.139 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.138 🚀 Python-3.9.6 torch-2.2.2 CPU (Intel Core(TM) i5-7500 3.40GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Dataset_1-9/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train12, nbs=64, nms=False, opset=None, optimize=False, op

train: Scanning /Users/valeriecowan/Desktop/Mathieu/Dataset_1-9/train/labels... 5409 images, 5409 backgrounds, 0 corrupt: 100%|██████████| 5409/5409 [00:06<00:00, 878.23it/s] 

train: New cache created: /Users/valeriecowan/Desktop/Mathieu/Dataset_1-9/train/labels.cache
WARNING ⚠️ Labels are missing or empty in /Users/valeriecowan/Desktop/Mathieu/Dataset_1-9/train/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


val: Fast image access ✅ (ping: 0.1±0.3 ms, read: 9.9±6.9 MB/s, size: 71.0 KB)


val: Scanning /Users/valeriecowan/Desktop/Mathieu/Dataset_1-9/valid/labels... 226 images, 226 backgrounds, 0 corrupt: 100%|██████████| 226/226 [00:00<00:00, 903.36it/s] 

val: New cache created: /Users/valeriecowan/Desktop/Mathieu/Dataset_1-9/valid/labels.cache
WARNING ⚠️ Labels are missing or empty in /Users/valeriecowan/Desktop/Mathieu/Dataset_1-9/valid/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


Plotting labels to runs/detect/train12/labels.jpg... 
WARNING ⚠️ zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train12
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/339 [00:00<?, ?it/s]


RuntimeError: Numpy is not available

In [ ]:
# Charger ton modèle fine-tuné (best.pt)
model = YOLO("runs/detect/train/weights/best.pt")

# Exporter en NCNN avec imgsz=320 pour Raspberry Pi
#model.export(format="ncnn", imgsz=320)
